In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# file_name
file_name = '/content/mock survey data 3.1.xlsx'


# Load the data
try:
    df_data = pd.read_excel(file_name, sheet_name ='data')
    print('Successfully loaded data sheet')
    df_dict = pd.read_excel(file_name, sheet_name='dictionary')
    print('Successfully loaded the dictionary sheet')
except ValueError:
    df_data = pd.read_excel(file_name, sheet_name =0)
    print('Loaded the first sheet')
    df_dict = pd.read_excel(file_name, sheet_name =1)
    print('Loaded the second sheet')

Successfully loaded data sheet
Successfully loaded the dictionary sheet


## Checking for Data Duplicate & Fixing

### Checking for non-unique rows

In [ ]:
# Define the columns to check for duplicates
# Exclude CASE column since I want to find rows that are identical in content
# even if they have different case IDs
cols_to_check = [c for c in df_data.columns if c!= 'case']

print(f"Number of duplicate rows: {df_data.duplicated(subset=cols_to_check).sum()}")

Number of duplicate rows: 8


### Doing a sense check on the data

In [ ]:
# Identify ALL duplicate sets
all_duplicates = df_data[df_data.duplicated(subset=cols_to_check, keep=False)].copy()

# Label the Rows to see which is the NEWER and OLDER row
is_old = all_duplicates.duplicated(subset=cols_to_check, keep='last')
all_duplicates['Status'] = 'New (Kept)'
all_duplicates.loc[is_old, 'Status'] = 'Old (Dropped)'

# Sort to pair identical records together
# Sorting by all check columns so that identical rows are next to each other
all_duplicates = all_duplicates.sort_values(by=cols_to_check)

# Reorder columns for readability (Status and Case first)
cols = ['Status', 'case'] + [c for c in all_duplicates.columns if c not in ['Status', 'case']]
comparison_table = all_duplicates[cols]

# Display the result
display(comparison_table)

,Status,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
947,Old (Dropped),952.00,2018,1,China,Others,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",1536.80,1.00,...,0,0,0,0,0,0,0,0,0,1
1165,New (Kept),1170.00,2018,1,China,Others,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",1536.80,1.00,...,0,0,0,0,0,0,0,0,0,1
32,Old (Dropped),33.00,2018,1,China,Shanghai,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",1536.80,1.00,...,0,0,0,0,0,0,0,0,0,1
955,Old (Dropped),960.00,2018,1,China,Shanghai,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",1536.80,1.00,...,0,0,0,0,0,0,0,0,0,1
1072,New (Kept),1077.00,2018,1,China,Shanghai,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",1536.80,1.00,...,0,0,0,0,0,0,0,0,0,1
319,Old (Dropped),322.00,2018,1,China,Shanghai,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",1536.80,1.00,...,0,0,0,0,0,0,0,0,0,1
954,New (Kept),959.00,2018,1,China,Shanghai,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",1536.80,1.00,...,0,0,0,0,0,0,0,0,0,1
318,Old (Dropped),321.00,2018,1,China,Shanghai,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",3910.41,1.00,...,0,0,0,0,0,0,0,0,0,1
1071,New (Kept),1076.00,2018,1,China,Shanghai,Leisure,"Others - Personal (e.g. weddings, funerals, etc)",3910.41,1.00,...,0,0,0,0,0,0,0,0,0,1
122,Old (Dropped),123.00,2018,1,China,Shanghai,NaN,"Others - Personal (e.g. weddings, funerals, etc)",1526.09,1.00,...,0,0,0,0,0,0,0,0,0,1


### Keep latest record and drop the others based on case ID

In [ ]:
# Remove duplicateste and keep the latest only
# keep = last ensure the row that appear last in the dataframe will be preserved
df_clean = df_data.drop_duplicates(subset=cols_to_check, keep='last').copy()

# Verification
print(f"Original Row Count: {len(df_data)}")
print(f"Duplicate rows removed: {len(df_data) - len(df_clean)}")
print(f"Final Row Count: {len(df_clean)}")

Original Row Count: 22974
Duplicate rows removed: 8
Final Row Count: 22966


## Checking for NULL / Empty in Case Column

In [ ]:
# Check for NULL values in Case Column
null_mask = df_clean['case'].isnull()
num_nulls = null_mask.sum()
print(f"Found {num_nulls} rows with NULL case IDs.")

# Display the rows to ensure they contain valid data
print("Rows with NULL case IDs:")
display(df_clean[null_mask])

Found 2 rows with NULL case IDs.
Rows with NULL case IDs:


,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
861,NaN,2018,1,Indonesia,Jakarta,Healthcare + Accompanying Pax,Accompanying a healthcare/ medical visitor for...,496.03,1.00,NaN,...,0,0,0,1,0,0,0,0,0,0
1723,NaN,2018,1,Indonesia,Jakarta,Healthcare + Accompanying Pax,In-patient (hospitalization) treatment,657.74,3.00,NaN,...,0,1,1,0,0,0,0,0,0,0


### Assigning CASE IDs to the unique rows without CASE ID

In [ ]:
if num_nulls > 0:
  # Calculate the starting ID (Max + 1)
  max_id = df_clean['case'].max()
  start_id = int(max_id) + 1

  # Generate new IDs
  new_ids = range(start_id, start_id + num_nulls)
  # Assign the IDs
  df_clean.loc[null_mask, 'case'] = new_ids

  # Convert to INT to remove the .0 behind
  df_clean['case'] = df_clean['case'].astype(int)

  print(f"Assigned new IDs: {list(new_ids)}")

# Display the result
if num_nulls > 0:
  display(df_clean[df_clean['case'].isin(new_ids)])

Assigned new IDs: [22984, 22985]


,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
861,22984,2018,1,Indonesia,Jakarta,Healthcare + Accompanying Pax,Accompanying a healthcare/ medical visitor for...,496.03,1.00,NaN,...,0,0,0,1,0,0,0,0,0,0
1723,22985,2018,1,Indonesia,Jakarta,Healthcare + Accompanying Pax,In-patient (hospitalization) treatment,657.74,3.00,NaN,...,0,1,1,0,0,0,0,0,0,0


## Identify missing values for cleaning

In [ ]:
# Calculate missing count & percentage

# Count cells where the value is missing (NaN/None)
missing_count = df_clean.isnull().sum()

# Count % of missing values per column
missing_percent = (df_clean.isnull().mean() * 100).round(2)

# Creating a dataframe for summarizing the missing counts and % per col
missing_data = pd.DataFrame({
    'Missing Count': missing_count,
    'Percentage (%)': missing_percent
})

# Keep only cols where at least 1 valye is missing then sort in desc order
missing_data = missing_data[missing_data['Missing Count'] > 0].sort_values(by='Percentage (%)', ascending=False)

In [ ]:
# Categorize columns with missing data

# Cols with high missingness (more than 90% missing data)
high_missing_cols = missing_data[missing_data['Percentage (%)'] > 90].index.tolist()

# Cols with missing data that are numeric in nature
numeric_cols = missing_data.index.intersection(df_clean.select_dtypes(include='number').columns).tolist()

# Cols with missing data that are categorical
categorical_cols = missing_data.index.intersection(df_clean.select_dtypes(include='object').columns).tolist()

# Display the result
print(f"Total Columns with Missing Data: {len(missing_data)}")
try:
    print(missing_data.to_markdown())
except:
    print(missing_data)

print(f"DROP (High Missing > 90%): {len(high_missing_cols)} columns")
print(f"   {high_missing_cols}")

print(f"\n FILL 0 (Numeric): {len(numeric_cols)} columns")
print(f"   {numeric_cols}")

print(f"\n FILL 'Unknown' (Categorical): {len(categorical_cols)} columns")
print(f"   {categorical_cols}")

Total Columns with Missing Data: 14
|                         |   Missing Count |   Percentage (%) |
|:------------------------|----------------:|-----------------:|
| Designation (free text) |           22774 |            99.16 |
| Sea_Terminal            |           20585 |            89.63 |
| Land_Terminal           |           20351 |            88.61 |
| Other Designation       |           20177 |            87.86 |
| Occupation.1            |           20176 |            87.85 |
| MainHotel               |            6682 |            29.1  |
| Air_Terminal            |            4997 |            21.76 |
| Purpose of Visit        |            2331 |            10.15 |
| shopping_any            |            1756 |             7.65 |
| shopping_others         |            1756 |             7.65 |
| shopping_fash           |              85 |             0.37 |
| City of Residence       |              12 |             0.05 |
| shopping_food           |               6 |         

### Fill numerical cols with 0

In [ ]:
# Identify the columns that are related to number
numeric_cols = [c for c in df_clean.columns if 'shopping' in c]

# Fill these columns with '0'
df_clean[numeric_cols] = df_clean[numeric_cols].fillna(0)
print(f" Filled NaNs with 0 in {len(numeric_cols)} numeric columns")

 Filled NaNs with 0 in 11 numeric columns


### Fill categorical cols with Unknown

In [ ]:
# Identify columns that are categorical
categorical_cols = ['Other Designation', 'Occupation.1', 'Purpose of Visit', 'City of Residence']

cols_to_fill = [c for c in categorical_cols if c in df_clean.columns]

# Fill these columns with 'Unknown'
df_clean[cols_to_fill] = df_clean[cols_to_fill].fillna('Unknown')
print(f" Filled NaNs with Unknown in {len(cols_to_fill)} categorical columns")

 Filled NaNs with Unknown in 4 categorical columns


## Checking of Whitespace

### Which columns have whitespace

In [ ]:
# Identify all text columns
str_cols = df_clean.select_dtypes(include=['object']).columns

# Identify which of these have trailing whitespace
dirty_cols = [c for c in str_cols if (df_clean[c].str.len() != df_clean[c].str.strip().str.len()).any()]
if dirty_cols:
    print(f"Check Columns with Trailing Whitespace: {dirty_cols}\n")
else:
    print(f"Clean no whitespace")

Check Columns with Trailing Whitespace: ['Sea_Terminal', 'Land_Terminal', 'Designation (free text)']



# Some other Sanity Check

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Checking for Negative Value

In [ ]:
# Select numeric columns
numeric_cols = df_clean.select_dtypes(include=['number']).columns

# Filter for rows with negative value
negative_rows = df_clean[(df_clean[numeric_cols] < 0).any(axis=1)]

if not negative_rows.empty:
    print(f"Found {len(negative_rows)} rows with negative values.")

    # Check which columns have the issue
    neg_cols = [c for c in numeric_cols if (df_clean[c] < 0).any()]

    for col in neg_cols:
        # Get unique negative values to spot patterns
        unique_negs = df_clean.loc[df_clean[col] < 0, col].unique()
        print(f"\nColumn '{col}' has these negative values: {unique_negs}")

        # Show sample row
        print(f"Sample Row")
        display(df_clean[df_clean[col] < 0][['case', col]].head(10))
else:
    print("No negative values found")

Found 2210 rows with negative values.

Column 'tototh' has these negative values: [-886.63 -212.23  -56.7  ... -554.13 -741.27 -679.04]
Sample Row


,case,tototh
23,24,-886.63
41,42,-212.23
61,62,-56.70
80,81,-221.65
96,97,-56.70
99,100,-26.70
107,108,-93.25
108,109,-291.38
115,116,-59.77
143,144,-246.72


### Turn Negative to +

In [ ]:
# Column with negative value
col = 'tototh'

# Use absolute to force it to become positive
df_clean[col] = df_clean[col].abs()
print(f"Converted negative values in '{col}' to positive")

Converted negative values in 'tototh' to positive


## Shopping

### Verify that the shopping expenditure matches the total shoping column

In [ ]:
# Indicate all the SHOPPING columns
shopping_cols = [
    'shopping_fash', 'shopping_jewllery', 'shopping_watches', 'shopping_wellness',
    'shopping_food', 'shopping_gifts', 'shopping_consumertech', 'shopping_antiques',
    'shopping_others', 'shopping_any'
    ]

# Sum horizontally across each row to get the total shopping spend for each visitor
df_clean['totshopping_corrected'] = df_clean[shopping_cols].sum(axis=1)

# Column for showing the difference between what is calculated and what is indicated for shopping
df_clean['diff_shopping'] = df_clean['totshopping'] - df_clean['totshopping_corrected']

### Create a new flag column to flag out mismatch in shopping

In [ ]:
# Create a new column for flagging the mismatch
# When the absolute difference is more than $1 set it as TRUE
df_clean['shopping_mismatch'] = (df_clean['totshopping'] - df_clean['totshopping_corrected']).abs() > 1.0

# Display the result
display(df_clean.head(10))

,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others,totshopping_corrected,diff_shopping,shopping_mismatch
0,1,2018,1,India,Delhi,Leisure,Holiday/ Rest & Relax,660.05,2.00,NaN,...,0,0,0,0,0,0,0,102.00,0.00,False
1,2,2018,1,India,Delhi,Leisure,Visiting friends/ relatives (who are not inter...,433.22,2.00,NaN,...,0,0,0,0,0,0,0,0.00,28.00,True
2,3,2018,1,United Arab Emirates,Dubai,Leisure,General business purpose,335.12,2.00,NaN,...,0,0,0,0,0,0,0,4076.00,0.00,False
3,4,2018,1,India,Others,Leisure,Holiday/ Rest & Relax,490.98,2.00,NaN,...,0,0,0,0,0,0,0,278.00,0.00,False
4,5,2018,1,Canada,Toronto,Business + Accompanying Pax,Visiting an international student (relative or...,379.80,2.00,NaN,...,0,0,0,0,0,0,0,2072.00,0.00,False
5,6,2018,1,China,Tianjin,Leisure,Accompanying an international student (relativ...,966.68,1.00,NaN,...,0,0,0,0,0,0,0,854.00,0.00,False
6,7,2018,1,United States,Others,Leisure,Visiting friends/ relatives (who are not inter...,419.70,1.00,NaN,...,0,0,0,0,0,0,0,96.00,-0.00,False
7,8,2018,1,Germany,Munich,Leisure,Holiday/ Rest & Relax,741.70,1.00,NaN,...,0,0,0,0,0,0,0,0.00,10.00,True
8,9,2018,1,Germany,Munich,Leisure,Visiting friends/ relatives (who are not inter...,1165.60,1.00,NaN,...,0,0,0,0,0,0,0,0.00,10.00,True
9,10,2018,1,Mexico,Monterrey,Leisure,Visiting friends/ relatives (who are not inter...,910.00,1.00,NaN,...,0,0,0,0,0,0,0,0.00,94.00,True


## Expenditure

### Verify the various expenditure columns

In [ ]:
# Indicate all the EXPENDITURE COLS
exp_cols = ['totacc', 'totfnb', 'tottran', 'totbiz', 'totedu', 'totmedi', 'tototh', 'totshopping']

# Sum horizontally across each row to get the total expenditure spend for each visitor
df_clean['totexp_corrected'] = df_clean[exp_cols].sum(axis=1)

# Column for showing the difference between what is calculated and what is indicated for shopping
df_clean['diff_expenditure'] = df_clean['tot.exp'] - df_clean['totexp_corrected']

### Create Flag column to flag out mismatch in EXPENDITURE

In [ ]:
# Create a new column for flagging the mismatch
# When the absolute difference is more than $1 set it as TRUE
df_clean['expenditure_mismatch'] = (df_clean['tot.exp'] - df_clean['totexp_corrected']).abs() > 1.0

# Display the result
display(df_clean.head(10))

,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others,totshopping_corrected,diff_shopping,shopping_mismatch,totexp_corrected,diff_expenditure,expenditure_mismatch
0,1,2018,1,India,Delhi,Leisure,Holiday/ Rest & Relax,660.05,2.00,NaN,...,0,0,0,0,102.00,0.00,False,1481.20,-0.30,False
1,2,2018,1,India,Delhi,Leisure,Visiting friends/ relatives (who are not inter...,433.22,2.00,NaN,...,0,0,0,0,0.00,28.00,True,100.40,-1.30,True
2,3,2018,1,United Arab Emirates,Dubai,Leisure,General business purpose,335.12,2.00,NaN,...,0,0,0,0,4076.00,0.00,False,5287.52,-1.25,True
3,4,2018,1,India,Others,Leisure,Holiday/ Rest & Relax,490.98,2.00,NaN,...,0,0,0,0,278.00,0.00,False,675.80,-1.30,True
4,5,2018,1,Canada,Toronto,Business + Accompanying Pax,Visiting an international student (relative or...,379.80,2.00,NaN,...,0,0,0,0,2072.00,0.00,False,4183.80,-1.25,True
5,6,2018,1,China,Tianjin,Leisure,Accompanying an international student (relativ...,966.68,1.00,NaN,...,0,0,0,0,854.00,0.00,False,990.16,-1.17,True
6,7,2018,1,United States,Others,Leisure,Visiting friends/ relatives (who are not inter...,419.70,1.00,NaN,...,0,0,0,0,96.00,-0.00,False,384.74,-0.66,False
7,8,2018,1,Germany,Munich,Leisure,Holiday/ Rest & Relax,741.70,1.00,NaN,...,0,0,0,0,0.00,10.00,True,188.18,-1.49,True
8,9,2018,1,Germany,Munich,Leisure,Visiting friends/ relatives (who are not inter...,1165.60,1.00,NaN,...,0,0,0,0,0.00,10.00,True,145.89,-1.49,True
9,10,2018,1,Mexico,Monterrey,Leisure,Visiting friends/ relatives (who are not inter...,910.00,1.00,NaN,...,0,0,0,0,0.00,94.00,True,128.17,-1.62,True


## Length of Stay vs Accoms

### Length of Stay vs Accommodation Spend


In [ ]:
# Find those rows where length of stay is 0 and accommodation spend is more than 0
mismatch_mask = (df_clean['Length of Stay'] == 0) & (df_clean['totacc'] > 0)

print(f"Found {mismatch_mask.sum()} rows with mismatched accommodation data")
cols_to_show = ['case', 'Length of Stay', 'totacc', 'Travel Type']
display(df_clean[mismatch_mask][cols_to_show].head(10))

Found 119 rows with mismatched accommodation data


,case,Length of Stay,totacc,Travel Type
1106,1111,0,54.98,Business (Non-packaged)
1401,1407,0,29.57,Non-packaged
2656,8326,0,54.98,Business (Non-packaged)
2873,8543,0,33.72,Business (Non-packaged)
2877,8547,0,54.98,Business (Non-packaged)
2881,8551,0,41.96,Business (Non-packaged)
3752,9422,0,47.50,Business (Non-packaged)
8584,14254,0,96.45,Packaged
8585,14255,0,65.58,Non-packaged
8586,14256,0,590.41,Packaged


### Creating flag column for accoms mismatch

In [ ]:
df_clean['Accommodation_Mismatch'] = mismatch_mask

display(df_clean.head(10))

,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others,totshopping_corrected,diff_shopping,shopping_mismatch,totexp_corrected,diff_expenditure,expenditure_mismatch,Accommodation_Mismatch
0,1,2018,1,India,Delhi,Leisure,Holiday/ Rest & Relax,660.05,2.00,NaN,...,0,0,0,102.00,0.00,False,1481.20,-0.30,False,False
1,2,2018,1,India,Delhi,Leisure,Visiting friends/ relatives (who are not inter...,433.22,2.00,NaN,...,0,0,0,0.00,28.00,True,100.40,-1.30,True,False
2,3,2018,1,United Arab Emirates,Dubai,Leisure,General business purpose,335.12,2.00,NaN,...,0,0,0,4076.00,0.00,False,5287.52,-1.25,True,False
3,4,2018,1,India,Others,Leisure,Holiday/ Rest & Relax,490.98,2.00,NaN,...,0,0,0,278.00,0.00,False,675.80,-1.30,True,False
4,5,2018,1,Canada,Toronto,Business + Accompanying Pax,Visiting an international student (relative or...,379.80,2.00,NaN,...,0,0,0,2072.00,0.00,False,4183.80,-1.25,True,False
5,6,2018,1,China,Tianjin,Leisure,Accompanying an international student (relativ...,966.68,1.00,NaN,...,0,0,0,854.00,0.00,False,990.16,-1.17,True,False
6,7,2018,1,United States,Others,Leisure,Visiting friends/ relatives (who are not inter...,419.70,1.00,NaN,...,0,0,0,96.00,-0.00,False,384.74,-0.66,False,False
7,8,2018,1,Germany,Munich,Leisure,Holiday/ Rest & Relax,741.70,1.00,NaN,...,0,0,0,0.00,10.00,True,188.18,-1.49,True,False
8,9,2018,1,Germany,Munich,Leisure,Visiting friends/ relatives (who are not inter...,1165.60,1.00,NaN,...,0,0,0,0.00,10.00,True,145.89,-1.49,True,False
9,10,2018,1,Mexico,Monterrey,Leisure,Visiting friends/ relatives (who are not inter...,910.00,1.00,NaN,...,0,0,0,0.00,94.00,True,128.17,-1.62,True,False


## Terminal

### Checking that either one of the terminal must be filled

In [ ]:
# Terminal cols
terminal_cols = ['Air_Terminal', 'Sea_Terminal', 'Land_Terminal']

# Check whether the terminal columns contain data or not
# e.g. if personal has Terminal 1 in air but NaN in Sea & Land then it's 1 mean it's correct
df_clean['Terminal_Count'] = df_clean[terminal_cols].notna().sum(axis=1)
print(df_clean['Terminal_Count'].value_counts())

Terminal_Count
1    22961
0        3
2        2
Name: count, dtype: int64


### Create Mismatch Terminal Flag where it's less than 1

In [ ]:
# Create flag when it's below 1
mismatch_mask = df_clean['Terminal_Count'] < 1
df_clean['Terminal_Mismatch'] = mismatch_mask

print(f"Found {mismatch_mask.sum()} rows with NO terminal info.")
cols_to_show = ['case', 'Terminal_Count', 'Terminal_Mismatch'] + terminal_cols
display(df_clean[mismatch_mask][cols_to_show].head(10))

Found 3 rows with NO terminal info.


,case,Terminal_Count,Terminal_Mismatch,Air_Terminal,Sea_Terminal,Land_Terminal
695,699,0,True,NaN,NaN,NaN
806,811,0,True,NaN,NaN,NaN
819,824,0,True,NaN,NaN,NaN


## Checking Expenditure Outliers

### Using IQR to get the bounds

In [ ]:
Q1 = df_clean['tot.exp'].quantile(0.25)
Q3 = df_clean['tot.exp'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1: {Q1}")
print(f"Q3: {Q3}")

Q1: 356.0775
Q3: 1280.3574999999998


### Create a flag for anything above the upper bound since they might be outliers

In [ ]:
# Create Flag
df_clean['Is_Expense_Outlier'] = df_clean['tot.exp'] > upper_bound

# Display
print(f"Found {df_clean['Is_Expense_Outlier'].sum()} expenditure outliers (> ${upper_bound:.2f}).")
cols_outlier = ['case', 'Country of Residence', 'tot.exp', 'Is_Expense_Outlier']
display(df_clean[df_clean['Is_Expense_Outlier']].sort_values('tot.exp', ascending=False)[cols_outlier].head())

Found 709 expenditure outliers (> $2666.78).


,case,Country of Residence,tot.exp,Is_Expense_Outlier
13910,19580,Philippines,29206.42,True
9860,15530,Japan,23791.22,True
2929,8599,Indonesia,15977.60,True
6392,12062,Indonesia,15218.74,True
1376,1382,India,11474.71,True


## Checking of Countries to see if there are any weird values in there

In [ ]:
# Extract unique countries
unique_countries = df_data['Country of Residence'].dropna().unique()

# Sort countries alphabetically
unique_countries.sort()

print(f"Total Unique Countries: {len(unique_countries)}")
print(unique_countries)

Total Unique Countries: 119
['Afghanistan' 'Algeria' 'Argentina' 'Australia' 'Austria' 'Azerbaijan'
 'Bahrain' 'Bangladesh' 'Belgium' 'Bosnia and Herzegovina' 'Brazil'
 'Brunei Darussalam' 'Cambodia' 'Canada' 'Chile' 'China' 'Colombia'
 'Cook Islands' 'Croatia' 'Cyprus' 'Czech Republic' 'Denmark' 'Egypt'
 'Equatorial Guinea' 'Estonia' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Georgia' 'Germany' 'Ghana' 'Greece' 'Guatemala' 'Haiti' 'Hong Kong'
 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq' 'Ireland' 'Israel'
 'Italy' 'Jamaica' 'Japan' 'Jordan' 'Kazakhstan' 'Kenya' 'Kuwait' 'Laos'
 'Latvia' 'Lebanon' 'Libya' 'Lithuania' 'Luxembourg' 'Macau' 'Madagascar'
 'Malawi' 'Malaysia' 'Maldives' 'Mauritania' 'Mauritius' 'Mexico' 'Monaco'
 'Mongolia' 'Morocco' 'Mozambique' 'Myanmar' 'Namibia' 'Nepal'
 'Netherlands' 'New Zealand' 'Nigeria' 'North Korea' 'Norway' 'Oman'
 'Others' 'Pakistan' 'Papua New Guinea' 'Peru' 'Philippines' 'Poland'
 'Portugal' 'Qatar' 'Reunion' 'Romania' 'Russia' 'Sao Tome

## Checking of Month Value is between 1 and 12

In [ ]:
# Identify rows where month is not between 1 - 12
invalid_months = df_clean[~df_clean['month'].between(1, 12)]

# Check if any errors found
if len(invalid_months) > 0:
    print(f"Found {len(invalid_months)} rows with invalid months:")
    print(invalid_months['month'].unique())
else:
    print("Passed: All months are between 1 and 12.")

Passed: All months are between 1 and 12.


In [ ]:
display(df_clean)

,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,totshopping_corrected,diff_shopping,shopping_mismatch,totexp_corrected,diff_expenditure,expenditure_mismatch,Accommodation_Mismatch,Terminal_Count,Terminal_Mismatch,Is_Expense_Outlier
0,1,2018,1,India,Delhi,Leisure,Holiday/ Rest & Relax,660.05,2.00,NaN,...,102.00,0.00,False,1481.20,-0.30,False,False,1,False,False
1,2,2018,1,India,Delhi,Leisure,Visiting friends/ relatives (who are not inter...,433.22,2.00,NaN,...,0.00,28.00,True,100.40,-1.30,True,False,1,False,False
2,3,2018,1,United Arab Emirates,Dubai,Leisure,General business purpose,335.12,2.00,NaN,...,4076.00,0.00,False,5287.52,-1.25,True,False,1,False,True
3,4,2018,1,India,Others,Leisure,Holiday/ Rest & Relax,490.98,2.00,NaN,...,278.00,0.00,False,675.80,-1.30,True,False,1,False,False
4,5,2018,1,Canada,Toronto,Business + Accompanying Pax,Visiting an international student (relative or...,379.80,2.00,NaN,...,2072.00,0.00,False,4183.80,-1.25,True,False,1,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22969,7789,2018,12,Australia,Brisbane,Leisure,Holiday/ Rest & Relax,310.35,2.00,NaN,...,60.00,0.00,False,1400.78,-0.90,False,False,1,False,False
22970,7790,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,109.39,2.00,NaN,...,23.50,-1.50,True,550.20,-1.67,True,False,1,False,False
22971,7791,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,326.91,2.00,NaN,...,0.00,8.00,True,454.48,-0.17,False,False,1,False,False
22972,7792,2018,12,Hong Kong,Hong Kong,Business + Accompanying Pax,Gather information/ facts on the education ser...,820.62,2.00,NaN,...,0.00,8.00,True,297.59,-0.17,False,False,1,False,False
